# Tarea 2 - Limpieza y Transformación de Datos con Scrapy

Bibliotecas utilizadas

!pip install scrapy


!pip install ipykernel

!pip install webdriver-manager

!pip install nbconvert


In [1]:
import os

# Carpetas a crear
folders = [
    "datalake/1_LANDING_ZONE",
    "datalake/2_REFINED_ZONE",
    "datalake/3_CONSUMPTION_ZONE",   
]

# Crear carpetas
for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(f"Carpeta '{folder}' creada.")
    else:
        print(f"La carpeta '{folder}' ya existe.")


La carpeta 'datalake/1_LANDING_ZONE' ya existe.
La carpeta 'datalake/2_REFINED_ZONE' ya existe.
La carpeta 'datalake/3_CONSUMPTION_ZONE' ya existe.


## 1 - Scrapy Implementation

### Obtener informacion de las empresas

### Categoria 1 - Bancos

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re, json, os

# ----------------------------------
# Configuración de Chrome
# ----------------------------------
options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1280,2000")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 15)

URL = "https://es.trustpilot.com/categories/bank?sort=reviews_count"
driver.get(URL)

# Espera a que aparezcan las tarjetas de empresa
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[name="business-unit-card"]')))
empresas = driver.find_elements(By.CSS_SELECTOR, 'a[name="business-unit-card"]')[:10]

datos = []

# ----------------------------------
# Helpers
# ----------------------------------
def limpiar_texto(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "").strip())

def get_first_text(el, selector_list):
    """Prueba varios selectores y devuelve el primer innerText válido."""
    for sel in selector_list:
        try:
            x = el.find_element(By.CSS_SELECTOR, sel)
            txt = x.get_attribute("innerText") or x.text
            txt = limpiar_texto(txt)
            if txt:
                return txt
        except:
            continue
    return ""

def parse_puntuacion(txt: str):
    """
    Convierte '4,8' o '4.8' a float 4.8.
    Si llega '4,8/5' también funciona.
    """
    if not txt:
        return "N/A"
    m = re.search(r"(\d+[.,]\d+|\d+)", txt)
    if not m:
        return "N/A"
    num = m.group(1).replace(",", ".")
    try:
        return float(num)
    except:
        return "N/A"

# ----------------------------------
# Scraping
# ----------------------------------
for empresa in empresas:
    # --- NOMBRE ---
    nombre = get_first_text(
        empresa,
        [
            'p[class*="CDS_Typography_heading-"]',
            'p[class*="heading-"]',
            'p[class*="heading"]',
        ],
    )
    if not nombre:
        nombre = "N/A"

    # --- UBICACIÓN ---
    ubicacion = ""
    # 1) Contenedor actual de ubicación
    try:
        u = empresa.find_element(By.CSS_SELECTOR, 'div[class*="styles_businessLocation__"] p')
        ubicacion = limpiar_texto(u.get_attribute("innerText") or u.text)
    except:
        ubicacion = ""

    # 2) Fallback atributo data antiguo
    if not ubicacion:
        try:
            uel = empresa.find_element(By.CSS_SELECTOR, 'span[data-business-location-typography="true"]')
            ubicacion = limpiar_texto(uel.get_attribute("innerText") or uel.text)
        except:
            ubicacion = ""

    # 3) Heurística general como último recurso
    if not ubicacion:
        try:
            candidatos = empresa.find_elements(
                By.CSS_SELECTOR,
                'p[class*="CDS_Typography_body-"], span[class*="CDS_Typography_body-"], p[dir="auto"]'
            )
            for c in candidatos:
                t = limpiar_texto(c.get_attribute("innerText") or c.text)
                if t and (
                    "," in t
                    or re.search(r"\d", t)
                    or re.search(r"(España|Spain|Madrid|Barcelona|Valencia|Sevilla|Bilbao)", t, re.I)
                ):
                    ubicacion = t
                    break
        except:
            pass

    if not ubicacion:
        ubicacion = "N/A"

    # --- PUNTUACIÓN ---
    puntuacion_txt = get_first_text(
        empresa,
        [
            'span[class*="styles_trustScore__"] span',
            'span[class*="styles_trustScore__"]',
            'span[weight="heavy"] span',
            'span[weight="heavy"]',
        ],
    )
    puntuacion = parse_puntuacion(puntuacion_txt)

    # --- PÁGINA WEB ---
    pagina_web = get_first_text(
        empresa,
        [
            'p[class*="styles_websiteUrlDisplayed__"]',
            'p[class*="styles_websiteUrlDisplayed"]',
            'p[class*="websiteUrl"]',
        ],
    )
    if pagina_web:
        pagina_web = pagina_web.replace(" ", "")
        if not pagina_web.lower().startswith(("http://", "https://")):
            pagina_web = "https://" + pagina_web
    else:
        pagina_web = "N/A"

    datos.append(
        {
            "nombre": nombre,
            "ubicacion": ubicacion,
            "puntuacion": puntuacion,
            "pagina_web": pagina_web,
        }
    )

driver.quit()

# ----------------------------------
# Guardar JSON
# ----------------------------------
out_path = "datalake/1_LANDING_ZONE/trustpilot_empresas_categoria1.json"
os.makedirs(os.path.dirname(out_path), exist_ok=True)

with open(out_path, "w", encoding="utf-8") as f:
    json.dump(datos, f, ensure_ascii=False, indent=4)

print("✅ Datos extraídos correctamente:")
print(json.dumps(datos, indent=4, ensure_ascii=False))


✅ Datos extraídos correctamente:
[
    {
        "nombre": "Aplazame",
        "ubicacion": "Calle Ulises 16-18, Madrid, España",
        "puntuacion": 4.8,
        "pagina_web": "https://aplazame.com"
    },
    {
        "nombre": "Bnext",
        "ubicacion": "Calle de Zurbano, 71, Madrid, España",
        "puntuacion": 1.5,
        "pagina_web": "https://www.bnext.es"
    },
    {
        "nombre": "CaixaBank",
        "ubicacion": "Calle Pintor Sorolla, 2-4, Valencia, España",
        "puntuacion": 1.2,
        "pagina_web": "https://www.caixabank.es"
    },
    {
        "nombre": "Crealsa",
        "ubicacion": "Carrer de Menorca 19, Planta 7ª (Edificio Aqua), València, España",
        "puntuacion": 4.4,
        "pagina_web": "https://www.crealsa.es"
    },
    {
        "nombre": "BBVA España",
        "ubicacion": "España",
        "puntuacion": 1.3,
        "pagina_web": "https://bbva.es"
    },
    {
        "nombre": "Banco Sabadell",
        "ubicacion": "España",
       

### Categoria 2 - Seguros de viajes

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re, json, os

# ----------------------------------
# Configuración de Chrome
# ----------------------------------
options = Options()
options.add_argument("--headless=new")     # headless moderno
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1280,2000")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 15)

URL = "https://es.trustpilot.com/categories/travel_insurance_company?sort=reviews_count"
driver.get(URL)

# Espera a que aparezcan las tarjetas de empresa
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[name="business-unit-card"]')))
empresas = driver.find_elements(By.CSS_SELECTOR, 'a[name="business-unit-card"]')[:10]  # top N
datos = []

# ----------------------------------
# Helpers
# ----------------------------------
def limpiar_texto(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "").strip())

def get_first_text(el, selector_list):
    """Prueba varios selectores y devuelve el primer innerText válido."""
    for sel in selector_list:
        try:
            x = el.find_element(By.CSS_SELECTOR, sel)
            txt = x.get_attribute("innerText") or x.text
            txt = limpiar_texto(txt)
            if txt:
                return txt
        except:
            continue
    return ""

def parse_puntuacion(txt: str):
    """
    Convierte '4,8' o '4.8' a float 4.8.
    Soporta '4,8/5' o con saltos de línea.
    """
    if not txt:
        return "N/A"
    m = re.search(r"(\d+[.,]\d+|\d+)", txt)
    if not m:
        return "N/A"
    num = m.group(1).replace(",", ".")
    try:
        return float(num)
    except:
        return "N/A"

# ----------------------------------
# Scraping
# ----------------------------------
for empresa in empresas:
    # --- NOMBRE ---
    nombre = get_first_text(
        empresa,
        [
            'p[class*="CDS_Typography_heading-"]',
            'p[class*="heading-"]',
            'p[class*="heading"]',
        ],
    ) or "N/A"

    # --- UBICACIÓN ---
    ubicacion = ""
    # 1) Contenedor típico de ubicación
    try:
        u = empresa.find_element(By.CSS_SELECTOR, 'div[class*="styles_businessLocation__"] p')
        ubicacion = limpiar_texto(u.get_attribute("innerText") or u.text)
    except:
        ubicacion = ""

    # 2) Fallback antiguo por atributo data
    if not ubicacion:
        try:
            uel = empresa.find_element(By.CSS_SELECTOR, 'span[data-business-location-typography="true"]')
            ubicacion = limpiar_texto(uel.get_attribute("innerText") or uel.text)
        except:
            ubicacion = ""

    # 3) Fallback genérico (por si cambian el DOM)
    if not ubicacion:
        try:
            candidatos = empresa.find_elements(
                By.CSS_SELECTOR,
                'p[class*="CDS_Typography_body-"], span[class*="CDS_Typography_body-"], p[dir="auto"]'
            )
            for c in candidatos:
                t = limpiar_texto(c.get_attribute("innerText") or c.text)
                # acepta país/ciudad o textos con coma/número
                if t and (
                    "," in t
                    or re.search(r"\d", t)
                    or re.search(r"(España|Spain|Madrid|Barcelona|Valencia|Sevilla|Bilbao)", t, re.I)
                ):
                    ubicacion = t
                    break
        except:
            pass
    if not ubicacion:
        ubicacion = "N/A"

    # --- PUNTUACIÓN ---
    puntuacion_txt = get_first_text(
        empresa,
        [
            'span[class*="styles_trustScore__"] span',
            'span[class*="styles_trustScore__"]',
            'span[weight="heavy"] span',
            'span[weight="heavy"]',
        ],
    )
    puntuacion = parse_puntuacion(puntuacion_txt)

    # --- PÁGINA WEB ---
    pagina_web = get_first_text(
        empresa,
        [
            'p[class*="styles_websiteUrlDisplayed__"]',
            'p[class*="styles_websiteUrlDisplayed"]',
            'p[class*="websiteUrl"]',
        ],
    )
    if pagina_web:
        pagina_web = pagina_web.replace(" ", "")
        if not pagina_web.lower().startswith(("http://", "https://")):
            pagina_web = "https://" + pagina_web
    else:
        pagina_web = "N/A"

    datos.append(
        {
            "nombre": nombre,
            "ubicacion": ubicacion,
            "puntuacion": puntuacion,
            "pagina_web": pagina_web,
        }
    )

driver.quit()

# ----------------------------------
# Guardar JSON
# ----------------------------------
out_path = "datalake/1_LANDING_ZONE/trustpilot_empresas_categoria2.json"
os.makedirs(os.path.dirname(out_path), exist_ok=True)

with open(out_path, "w", encoding="utf-8") as f:
    json.dump(datos, f, ensure_ascii=False, indent=4)

print("✅ Datos extraídos correctamente:")
print(json.dumps(datos, indent=4, ensure_ascii=False))


✅ Datos extraídos correctamente:
[
    {
        "nombre": "Heymondo Seguros de Viaje",
        "ubicacion": "Calle Alaba, número 140, 2º 4ª, Barcelona, España",
        "puntuacion": 4.5,
        "pagina_web": "https://heymondo.es"
    },
    {
        "nombre": "Intermundial Seguros de viaje",
        "ubicacion": "Calle de Irún, 7, Madrid, España",
        "puntuacion": 4.5,
        "pagina_web": "https://intermundial.es"
    },
    {
        "nombre": "IATI Seguros",
        "ubicacion": "Avinguda Diagonal 622, Barcelona, España",
        "puntuacion": 4.4,
        "pagina_web": "https://iatiseguros.com"
    },
    {
        "nombre": "Abbeygate Seguros España",
        "ubicacion": "Ctra Nacional 340, KM148.5, Estepona, España",
        "puntuacion": 4.8,
        "pagina_web": "https://www.abbeygateinsure.com"
    },
    {
        "nombre": "MAPFRE España",
        "ubicacion": "Carretera de Pozuelo 52, Majadahonda, España",
        "puntuacion": 1.5,
        "pagina_web": "https:

### Categoria 3 - Concesionario de autos

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re, json, os

# ----------------------------------
# Configuración de Chrome
# ----------------------------------
options = Options()
options.add_argument("--headless=new")     # headless moderno
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1280,2000")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 15)

URL = "https://es.trustpilot.com/categories/car_dealer?sort=reviews_count"
driver.get(URL)

# Espera a que aparezcan las tarjetas de empresa
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[name="business-unit-card"]')))
empresas = driver.find_elements(By.CSS_SELECTOR, 'a[name="business-unit-card"]')[:10]  # top N
datos = []

# ----------------------------------
# Helpers
# ----------------------------------
def limpiar_texto(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "").strip())

def get_first_text(el, selector_list):
    """Prueba varios selectores y devuelve el primer innerText válido."""
    for sel in selector_list:
        try:
            x = el.find_element(By.CSS_SELECTOR, sel)
            txt = x.get_attribute("innerText") or x.text
            txt = limpiar_texto(txt)
            if txt:
                return txt
        except:
            continue
    return ""

def parse_puntuacion(txt: str):
    """
    Convierte '4,8' o '4.8' a float 4.8.
    Soporta '4,8/5' o con saltos de línea.
    """
    if not txt:
        return "N/A"
    m = re.search(r"(\d+[.,]\d+|\d+)", txt)
    if not m:
        return "N/A"
    num = m.group(1).replace(",", ".")
    try:
        return float(num)
    except:
        return "N/A"

# ----------------------------------
# Scraping
# ----------------------------------
for empresa in empresas:
    # --- NOMBRE ---
    nombre = get_first_text(
        empresa,
        [
            'p[class*="CDS_Typography_heading-"]',
            'p[class*="heading-"]',
            'p[class*="heading"]',
        ],
    ) or "N/A"

    # --- UBICACIÓN ---
    ubicacion = ""
    # 1) Contenedor típico de ubicación
    try:
        u = empresa.find_element(By.CSS_SELECTOR, 'div[class*="styles_businessLocation__"] p')
        ubicacion = limpiar_texto(u.get_attribute("innerText") or u.text)
    except:
        ubicacion = ""

    # 2) Fallback atributo data antiguo
    if not ubicacion:
        try:
            uel = empresa.find_element(By.CSS_SELECTOR, 'span[data-business-location-typography="true"]')
            ubicacion = limpiar_texto(uel.get_attribute("innerText") or uel.text)
        except:
            ubicacion = ""

    # 3) Fallback genérico (por si cambian el DOM)
    if not ubicacion:
        try:
            candidatos = empresa.find_elements(
                By.CSS_SELECTOR,
                'p[class*="CDS_Typography_body-"], span[class*="CDS_Typography_body-"], p[dir="auto"]'
            )
            for c in candidatos:
                t = limpiar_texto(c.get_attribute("innerText") or c.text)
                # acepta país/ciudad o textos con coma/número
                if t and (
                    "," in t
                    or re.search(r"\d", t)
                    or re.search(r"(España|Spain|Madrid|Barcelona|Valencia|Sevilla|Bilbao)", t, re.I)
                ):
                    ubicacion = t
                    break
        except:
            pass
    if not ubicacion:
        ubicacion = "N/A"

    # --- PUNTUACIÓN ---
    puntuacion_txt = get_first_text(
        empresa,
        [
            'span[class*="styles_trustScore__"] span',
            'span[class*="styles_trustScore__"]',
            'span[weight="heavy"] span',
            'span[weight="heavy"]',
        ],
    )
    puntuacion = parse_puntuacion(puntuacion_txt)

    # --- PÁGINA WEB ---
    pagina_web = get_first_text(
        empresa,
        [
            'p[class*="styles_websiteUrlDisplayed__"]',
            'p[class*="styles_websiteUrlDisplayed"]',
            'p[class*="websiteUrl"]',
        ],
    )
    if pagina_web:
        pagina_web = pagina_web.replace(" ", "")
        if not pagina_web.lower().startswith(("http://", "https://")):
            pagina_web = "https://" + pagina_web
    else:
        pagina_web = "N/A"

    datos.append(
        {
            "nombre": nombre,
            "ubicacion": ubicacion,
            "puntuacion": puntuacion,
            "pagina_web": pagina_web,
        }
    )

driver.quit()

# ----------------------------------
# Guardar JSON
# ----------------------------------
out_path = "datalake/1_LANDING_ZONE/trustpilot_empresas_categoria3.json"
os.makedirs(os.path.dirname(out_path), exist_ok=True)

with open(out_path, "w", encoding="utf-8") as f:
    json.dump(datos, f, ensure_ascii=False, indent=4)

print("✅ Datos extraídos correctamente:")
print(json.dumps(datos, indent=4, ensure_ascii=False))


✅ Datos extraídos correctamente:
[
    {
        "nombre": "compramostucoche.es",
        "ubicacion": "España",
        "puntuacion": 4.6,
        "pagina_web": "https://www.compramostucoche.es"
    },
    {
        "nombre": "OcasionPlus",
        "ubicacion": "Avenida Juan Carlos I 30, Collado Villalba, España",
        "puntuacion": 4.1,
        "pagina_web": "https://ocasionplus.com"
    },
    {
        "nombre": "Autohero España",
        "ubicacion": "Calle Rosario Pino 14-16 Planta 1, Madrid, España",
        "puntuacion": 4.2,
        "pagina_web": "https://autohero.com/es"
    },
    {
        "nombre": "Carwow ES",
        "ubicacion": "C. de Serrano Anguita, 13, Madrid, España",
        "puntuacion": 4.4,
        "pagina_web": "https://www.carwow.es"
    },
    {
        "nombre": "Carplus España",
        "ubicacion": "España",
        "puntuacion": 4.6,
        "pagina_web": "https://carplus.es"
    },
    {
        "nombre": "Hr Motor",
        "ubicacion": "España",
  

### Scrapping de las reviews de las empresas

### Categoria 1

Scrap - Paginado

In [6]:
import time
import json
import pandas as pd
import os
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ------------- CONFIG -------------
DEBUG = False                 # True = imprime y guarda evidencia (screenshot + html)
MAX_PAGES = 5               # máximo de páginas por empresa
EMPRESAS_JSON = 'datalake/1_LANDING_ZONE/trustpilot_empresas_categoria1.json'
DATASET_CSV   = 'datalake/1_LANDING_ZONE/dataset_categoria1.csv'
SALIDA_JSON   = 'datalake/1_LANDING_ZONE/reviews_trustpilot_empresas_categoria1.json'

# ------------- HELPERS -------------
def norm_text(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "").strip())

def parse_estrellas_from_alt(alt_txt: str):
    """
    'Valorada con 4 estrellas sobre 5' -> '4'
    """
    if not alt_txt:
        return "N/A"
    m = re.search(r"(\d)\s*estrellas", alt_txt, flags=re.I)
    return m.group(1) if m else "N/A"

def normalizar_dominio(pagina_web: str) -> str:
    """
    Espera 'www.midominio.es' o 'midominio.es'.
    Quita http(s):// si viene, y slashes al final.
    """
    if not pagina_web:
        return ""
    pw = pagina_web.strip()
    pw = re.sub(r"^https?://", "", pw, flags=re.I)
    pw = pw.strip("/")
    return pw

def extraer_review_uid(card):
    """
    Intenta extraer el id de la reseña desde el href del título: '/reviews/<id>'
    Devuelve el segmento <id> o '' si no existe.
    """
    try:
        a = card.find_element(By.CSS_SELECTOR, 'a[data-review-title-typography="true"]')
        href = a.get_attribute("href")
        if href:
            m = re.search(r"/reviews/([^/?#]+)", href)
            if m:
                return m.group(1)
    except:
        pass
    return ""

def aceptar_cookies_si_aparecen(driver, wait):
    """
    Cierra la barra de cookies de Trustpilot si aparece (botón 'Entendido' o similar).
    """
    try:
        # botón con texto 'Entendido'
        btn = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, "//button[.//span[normalize-space(text())='Entendido'] or normalize-space(text())='Entendido']"))
        )
        btn.click()
        time.sleep(0.5)
        return True
    except:
        # otros textos típicos
        try:
            btn2 = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, "//button[normalize-space(text())='Aceptar']"))
            )
            btn2.click()
            time.sleep(0.5)
            return True
        except:
            return False

# ------------- CARGA EMPRESAS -------------
with open(EMPRESAS_JSON, 'r', encoding='utf-8') as f:
    empresas_data = json.load(f)

# ------------- CARGA / CREA DATASET -------------
if os.path.exists(DATASET_CSV):
    existing_df = pd.read_csv(DATASET_CSV)
else:
    existing_df = pd.DataFrame(columns=["id_reseña", "Fecha", "Título", "Contenido", "Empresa", "Calificación"])

existing_df['Fecha'] = pd.to_datetime(existing_df['Fecha'], errors='coerce')

# ------------- SELENIUM -------------
options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--window-size=1366,2400")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/118.0.5993.70 Safari/537.36"
)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 25)

resultados = []
reviews_by_company = {}

try:
    for empresa in empresas_data:
        nombre_empresa = empresa.get("nombre", "N/A")
        pagina_web = normalizar_dominio(empresa.get("pagina_web", ""))
        if not pagina_web:
            print(f"⚠️ Empresa sin 'pagina_web': {nombre_empresa}. Omitida.")
            continue

        print(f"\n🔍 Verificando nuevas reseñas de {nombre_empresa} ({pagina_web})...")

        # Histórico de esa empresa
        old_df = existing_df[existing_df["Empresa"] == nombre_empresa].copy()
        if not old_df.empty:
            old_df["Título"] = old_df["Título"].fillna("").str.strip()
            old_df["Fecha"] = pd.to_datetime(old_df["Fecha"], errors="coerce")

        reseñas_nuevas = []

        for page in range(1, MAX_PAGES + 1):
            review_url = f"https://es.trustpilot.com/review/{pagina_web}" + (f"?page={page}" if page > 1 else "")
            print(f"🌐 Accediendo a: {review_url}")
            driver.get(review_url)

            # Aceptar cookies si estorban
            aceptar_cookies_si_aparecen(driver, wait)

            # Espera a que al menos haya 1 card de reseña en la página
            try:
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-testid="service-review-card-v2"]')))
            except Exception as e:
                print(f"⚠️ No se encontraron cards en página {page}: {e}")
                # guarda para debug y rompe
                if DEBUG:
                    ss = f"_debug_reviews_{nombre_empresa}_p{page}_nocards.png"
                    driver.save_screenshot(ss)
                    with open(f"_debug_reviews_{nombre_empresa}_p{page}_nocards.html", "w", encoding="utf-8") as fh:
                        fh.write(driver.page_source)
                    print(f"💾 Guardé evidencia sin cards: {ss}")
                break

            # Forzar render con scrolls suaves (contenido lazy)
            for _ in range(3):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1.2)

            # Cards reales (BÚSQUEDA GLOBAL, no dentro de un contenedor específico)
            cards = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid="service-review-card-v2"]')

            if DEBUG:
                print(f"🔎 Encontradas {len(cards)} cards en página {page}")
                try:
                    ss_name = f"_debug_reviews_{nombre_empresa}_p{page}.png"
                    html_name = f"_debug_reviews_{nombre_empresa}_p{page}.html"
                    driver.save_screenshot(ss_name)
                    with open(html_name, "w", encoding="utf-8") as fh:
                        fh.write(driver.page_source)
                    print(f"💾 Guardé screenshot y HTML: {ss_name} / {html_name}")
                except Exception as e:
                    print(f"⚠️ No pude guardar debug files: {e}")

            nuevas_esta_pagina = []

            for i, card in enumerate(cards, start=1):
                try:
                    # Fecha (ISO en <time datetime="...">)
                    try:
                        time_el = card.find_element(By.CSS_SELECTOR, 'time[datetime]')
                        fecha = pd.to_datetime(time_el.get_attribute("datetime"), errors='coerce')
                    except:
                        fecha = pd.NaT

                    # Título
                    try:
                        h2 = card.find_element(By.CSS_SELECTOR, 'h2[data-service-review-title-typography="true"]')
                        titulo = norm_text(h2.get_attribute("innerText") or h2.text)
                    except:
                        titulo = "N/A"

                    # Contenido
                    try:
                        p = card.find_element(By.CSS_SELECTOR, 'p[data-service-review-text-typography="true"]')
                        contenido = norm_text(p.get_attribute("innerText") or p.text)
                    except:
                        contenido = "Sin contenido"

                    # Calificación (solo número)
                    try:
                        img = card.find_element(By.CSS_SELECTOR, 'img[alt*="Valorada con"]')
                        estrellas = parse_estrellas_from_alt(img.get_attribute("alt"))
                    except:
                        estrellas = "N/A"

                    # UID de la review (por si luego quieres deduplicar por id real)
                    review_uid = extraer_review_uid(card)

                    if DEBUG and i <= 3:
                        print(f"  • Card #{i}: fecha={fecha}, estrellas={estrellas}, título='{titulo[:60]}', uid='{review_uid}'")

                    # DEDUP: por compatibilidad, Título + Fecha
                    existe = (
                        not old_df[
                            (old_df["Título"].str.strip() == titulo) &
                            (old_df["Fecha"] == fecha)
                        ].empty
                    )

                    if not existe:
                        nuevas_esta_pagina.append({
                            "Empresa": nombre_empresa,
                            "Título": titulo,
                            "Contenido": contenido,
                            "Fecha": fecha,
                            "Calificación": estrellas
                        })

                except Exception as e_card:
                    if DEBUG:
                        print(f"  • Aviso: card con error: {e_card}")
                    continue

            if nuevas_esta_pagina:
                reseñas_nuevas.extend(nuevas_esta_pagina)
                print(f"✅ {len(nuevas_esta_pagina)} reseñas nuevas en página {page}. Continuando...")
            else:
                print(f"🛑 No hay reseñas nuevas en página {page}. Deteniendo scraping para esta empresa.")
                break

        # Consolidar por empresa
        if reseñas_nuevas:
            df_nuevas = pd.DataFrame(reseñas_nuevas)
            df_nuevas['Fecha'] = pd.to_datetime(df_nuevas['Fecha'], errors='coerce')
            combined_df = pd.concat([old_df, df_nuevas], ignore_index=True)
            combined_df.drop_duplicates(subset=["Título", "Fecha"], keep='first', inplace=True)
            combined_df.sort_values(by="Fecha", ascending=False, inplace=True)
            combined_df.reset_index(drop=True, inplace=True)
            combined_df.fillna("", inplace=True)

            # id_reseña por empresa (mantenemos tu formato)
            total = len(combined_df)
            combined_df["id_reseña"] = [f"{nombre_empresa.replace(' ', '')}_N{total - idx}" for idx in range(total)]

            reviews_by_company[nombre_empresa] = combined_df.to_dict(orient="records")
            resultados.append(combined_df)
        else:
            if not old_df.empty:
                reviews_by_company[nombre_empresa] = old_df.to_dict(orient="records")
            print(f"📭 No se detectaron nuevas reseñas para {nombre_empresa}.")

finally:
    driver.quit()

# ------------- SALIDA CSV -------------
if resultados:
    final_df = pd.concat([existing_df] + resultados, ignore_index=True)
    final_df.drop_duplicates(subset=["Título", "Fecha", "Empresa"], keep='first', inplace=True)
    final_df.fillna("", inplace=True)

    # Recalcular ids por empresa en orden de fecha desc
    final_df["Fecha"] = pd.to_datetime(final_df["Fecha"], errors="coerce")
    final_df.sort_values(by=["Empresa", "Fecha"], ascending=[True, False], inplace=True)

    final_df["id_reseña"] = final_df.groupby("Empresa").cumcount(ascending=False) + 1
    final_df["id_reseña"] = final_df.apply(
        lambda row: f"{row['Empresa'].replace(' ', '')}_N{row['id_reseña']}", axis=1
    )

    final_df = final_df[["id_reseña", "Fecha", "Título", "Contenido", "Empresa", "Calificación"]]

    # Ordenar por número de id descendente dentro de cada empresa
    final_df["id_num"] = final_df["id_reseña"].str.extract(r'_N(\d+)').astype(int)
    final_df.sort_values(by=["Empresa", "id_num"], ascending=[True, False], inplace=True)
    final_df.drop(columns=["id_num"], inplace=True)

    os.makedirs(os.path.dirname(DATASET_CSV), exist_ok=True)
    final_df.to_csv(DATASET_CSV, index=False, encoding="utf-8-sig")
    print(f"\n🎉 Dataset actualizado guardado en '{DATASET_CSV}'")
else:
    print("\n✅ No hubo actualizaciones, el dataset se mantiene igual.")

# ------------- SALIDA JSON (anidado por empresa) -------------
for empresa in empresas_data:
    nombre_empresa = empresa.get("nombre", "N/A")
    empresa["reseñas"] = reviews_by_company.get(nombre_empresa, [])

os.makedirs(os.path.dirname(SALIDA_JSON), exist_ok=True)
with open(SALIDA_JSON, "w", encoding="utf-8") as f:
    json.dump(empresas_data, f, ensure_ascii=False, indent=4, default=str)

print(f"📦 Archivo JSON guardado en '{SALIDA_JSON}'")



🔍 Verificando nuevas reseñas de Aplazame (aplazame.com)...
🌐 Accediendo a: https://es.trustpilot.com/review/aplazame.com
✅ 20 reseñas nuevas en página 1. Continuando...
🌐 Accediendo a: https://es.trustpilot.com/review/aplazame.com?page=2
✅ 20 reseñas nuevas en página 2. Continuando...
🌐 Accediendo a: https://es.trustpilot.com/review/aplazame.com?page=3
✅ 20 reseñas nuevas en página 3. Continuando...
🌐 Accediendo a: https://es.trustpilot.com/review/aplazame.com?page=4
✅ 20 reseñas nuevas en página 4. Continuando...
🌐 Accediendo a: https://es.trustpilot.com/review/aplazame.com?page=5
✅ 20 reseñas nuevas en página 5. Continuando...

🔍 Verificando nuevas reseñas de Bnext (www.bnext.es)...
🌐 Accediendo a: https://es.trustpilot.com/review/www.bnext.es
✅ 8 reseñas nuevas en página 1. Continuando...
🌐 Accediendo a: https://es.trustpilot.com/review/www.bnext.es?page=2
🛑 No hay reseñas nuevas en página 2. Deteniendo scraping para esta empresa.

🔍 Verificando nuevas reseñas de CaixaBank (www.cai

### Categoria 2

Scrap - Paginado

In [7]:
import time
import json
import pandas as pd
import os
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ------------- CONFIG -------------
DEBUG = False                 # True = imprime y guarda evidencia (screenshot + html)
MAX_PAGES = 5                 # máximo de páginas por empresa
EMPRESAS_JSON = 'datalake/1_LANDING_ZONE/trustpilot_empresas_categoria2.json'
DATASET_CSV   = 'datalake/1_LANDING_ZONE/dataset_categoria2.csv'
SALIDA_JSON   = 'datalake/1_LANDING_ZONE/reviews_trustpilot_empresas_categoria2.json'

# ------------- HELPERS -------------
def norm_text(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "").strip())

def parse_estrellas_from_alt(alt_txt: str):
    """
    'Valorada con 4 estrellas sobre 5' -> '4'
    """
    if not alt_txt:
        return "N/A"
    m = re.search(r"(\d)\s*estrellas", alt_txt, flags=re.I)
    return m.group(1) if m else "N/A"

def normalizar_dominio(pagina_web: str) -> str:
    """
    Espera 'www.midominio.es' o 'midominio.es'.
    Quita http(s):// si viene, y slashes al final.
    """
    if not pagina_web:
        return ""
    pw = pagina_web.strip()
    pw = re.sub(r"^https?://", "", pw, flags=re.I)
    pw = pw.strip("/")
    return pw

def extraer_review_uid(card):
    """
    Intenta extraer el id de la reseña desde el href del título: '/reviews/<id>'
    Devuelve el segmento <id> o '' si no existe.
    """
    try:
        a = card.find_element(By.CSS_SELECTOR, 'a[data-review-title-typography="true"]')
        href = a.get_attribute("href")
        if href:
            m = re.search(r"/reviews/([^/?#]+)", href)
            if m:
                return m.group(1)
    except:
        pass
    return ""

def aceptar_cookies_si_aparecen(driver, wait):
    """
    Cierra la barra de cookies de Trustpilot si aparece (botón 'Entendido' o similar).
    """
    try:
        btn = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, "//button[.//span[normalize-space(text())='Entendido'] or normalize-space(text())='Entendido']"))
        )
        btn.click()
        time.sleep(0.5)
        return True
    except:
        try:
            btn2 = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, "//button[normalize-space(text())='Aceptar']"))
            )
            btn2.click()
            time.sleep(0.5)
            return True
        except:
            return False

# ------------- CARGA EMPRESAS -------------
with open(EMPRESAS_JSON, 'r', encoding='utf-8') as f:
    empresas_data = json.load(f)

# ------------- CARGA / CREA DATASET -------------
if os.path.exists(DATASET_CSV):
    existing_df = pd.read_csv(DATASET_CSV)
else:
    existing_df = pd.DataFrame(columns=["id_reseña", "Fecha", "Título", "Contenido", "Empresa", "Calificación"])

existing_df['Fecha'] = pd.to_datetime(existing_df['Fecha'], errors='coerce')

# ------------- SELENIUM -------------
options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--window-size=1366,2400")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/118.0.5993.70 Safari/537.36"
)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 25)

resultados = []
reviews_by_company = {}

try:
    for empresa in empresas_data:
        nombre_empresa = empresa.get("nombre", "N/A")
        pagina_web = normalizar_dominio(empresa.get("pagina_web", ""))
        if not pagina_web:
            print(f"⚠️ Empresa sin 'pagina_web': {nombre_empresa}. Omitida.")
            continue

        print(f"\n🔍 Verificando nuevas reseñas de {nombre_empresa} ({pagina_web})...")

        # Histórico de esa empresa
        old_df = existing_df[existing_df["Empresa"] == nombre_empresa].copy()
        if not old_df.empty:
            old_df["Título"] = old_df["Título"].fillna("").str.strip()
            old_df["Fecha"] = pd.to_datetime(old_df["Fecha"], errors="coerce")

        reseñas_nuevas = []

        for page in range(1, MAX_PAGES + 1):
            review_url = f"https://es.trustpilot.com/review/{pagina_web}" + (f"?page={page}" if page > 1 else "")
            print(f"🌐 Accediendo a: {review_url}")
            driver.get(review_url)

            # Aceptar cookies si estorban
            aceptar_cookies_si_aparecen(driver, wait)

            # Espera a que al menos haya 1 card de reseña en la página
            try:
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-testid="service-review-card-v2"]')))
            except Exception as e:
                print(f"⚠️ No se encontraron cards en página {page}: {e}")
                if DEBUG:
                    ss = f"_debug_reviews_{nombre_empresa}_p{page}_nocards.png"
                    driver.save_screenshot(ss)
                    with open(f"_debug_reviews_{nombre_empresa}_p{page}_nocards.html", "w", encoding="utf-8") as fh:
                        fh.write(driver.page_source)
                    print(f"💾 Guardé evidencia sin cards: {ss}")
                break

            # Forzar render con scrolls suaves (contenido lazy)
            for _ in range(3):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1.2)

            # Cards reales (búsqueda global)
            cards = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid="service-review-card-v2"]')

            if DEBUG:
                print(f"🔎 Encontradas {len(cards)} cards en página {page}")
                try:
                    ss_name = f"_debug_reviews_{nombre_empresa}_p{page}.png"
                    html_name = f"_debug_reviews_{nombre_empresa}_p{page}.html"
                    driver.save_screenshot(ss_name)
                    with open(html_name, "w", encoding="utf-8") as fh:
                        fh.write(driver.page_source)
                    print(f"💾 Guardé screenshot y HTML: {ss_name} / {html_name}")
                except Exception as e:
                    print(f"⚠️ No pude guardar debug files: {e}")

            nuevas_esta_pagina = []

            for i, card in enumerate(cards, start=1):
                try:
                    # Fecha
                    try:
                        time_el = card.find_element(By.CSS_SELECTOR, 'time[datetime]')
                        fecha = pd.to_datetime(time_el.get_attribute("datetime"), errors='coerce')
                    except:
                        fecha = pd.NaT

                    # Título
                    try:
                        h2 = card.find_element(By.CSS_SELECTOR, 'h2[data-service-review-title-typography="true"]')
                        titulo = norm_text(h2.get_attribute("innerText") or h2.text)
                    except:
                        titulo = "N/A"

                    # Contenido
                    try:
                        p = card.find_element(By.CSS_SELECTOR, 'p[data-service-review-text-typography="true"]')
                        contenido = norm_text(p.get_attribute("innerText") or p.text)
                    except:
                        contenido = "Sin contenido"

                    # Calificación (solo número)
                    try:
                        img = card.find_element(By.CSS_SELECTOR, 'img[alt*="Valorada con"]')
                        estrellas = parse_estrellas_from_alt(img.get_attribute("alt"))
                    except:
                        estrellas = "N/A"

                    # UID de la review (opcional)
                    review_uid = extraer_review_uid(card)

                    if DEBUG and i <= 3:
                        print(f"  • Card #{i}: fecha={fecha}, estrellas={estrellas}, título='{titulo[:60]}', uid='{review_uid}'")

                    # DEDUP: Título + Fecha
                    existe = (
                        not old_df[
                            (old_df["Título"].str.strip() == titulo) &
                            (old_df["Fecha"] == fecha)
                        ].empty
                    )

                    if not existe:
                        nuevas_esta_pagina.append({
                            "Empresa": nombre_empresa,
                            "Título": titulo,
                            "Contenido": contenido,
                            "Fecha": fecha,
                            "Calificación": estrellas
                        })

                except Exception as e_card:
                    if DEBUG:
                        print(f"  • Aviso: card con error: {e_card}")
                    continue

            if nuevas_esta_pagina:
                reseñas_nuevas.extend(nuevas_esta_pagina)
                print(f"✅ {len(nuevas_esta_pagina)} reseñas nuevas en página {page}. Continuando...")
            else:
                print(f"🛑 No hay reseñas nuevas en página {page}. Deteniendo scraping para esta empresa.")
                break

        # Consolidar por empresa
        if reseñas_nuevas:
            df_nuevas = pd.DataFrame(reseñas_nuevas)
            df_nuevas['Fecha'] = pd.to_datetime(df_nuevas['Fecha'], errors='coerce')
            combined_df = pd.concat([old_df, df_nuevas], ignore_index=True)
            combined_df.drop_duplicates(subset=["Título", "Fecha"], keep='first', inplace=True)
            combined_df.sort_values(by="Fecha", ascending=False, inplace=True)
            combined_df.reset_index(drop=True, inplace=True)
            combined_df.fillna("", inplace=True)

            total = len(combined_df)
            combined_df["id_reseña"] = [f"{nombre_empresa.replace(' ', '')}_N{total - idx}" for idx in range(total)]

            reviews_by_company[nombre_empresa] = combined_df.to_dict(orient="records")
            resultados.append(combined_df)
        else:
            if not old_df.empty:
                reviews_by_company[nombre_empresa] = old_df.to_dict(orient="records")
            print(f"📭 No se detectaron nuevas reseñas para {nombre_empresa}.")

finally:
    driver.quit()

# ------------- SALIDA CSV -------------
if resultados:
    final_df = pd.concat([existing_df] + resultados, ignore_index=True)
    final_df.drop_duplicates(subset=["Título", "Fecha", "Empresa"], keep='first', inplace=True)
    final_df.fillna("", inplace=True)

    final_df["Fecha"] = pd.to_datetime(final_df["Fecha"], errors="coerce")
    final_df.sort_values(by=["Empresa", "Fecha"], ascending=[True, False], inplace=True)

    final_df["id_reseña"] = final_df.groupby("Empresa").cumcount(ascending=False) + 1
    final_df["id_reseña"] = final_df.apply(
        lambda row: f"{row['Empresa'].replace(' ', '')}_N{row['id_reseña']}", axis=1
    )

    final_df = final_df[["id_reseña", "Fecha", "Título", "Contenido", "Empresa", "Calificación"]]

    final_df["id_num"] = final_df["id_reseña"].str.extract(r'_N(\d+)').astype(int)
    final_df.sort_values(by=["Empresa", "id_num"], ascending=[True, False], inplace=True)
    final_df.drop(columns=["id_num"], inplace=True)

    os.makedirs(os.path.dirname(DATASET_CSV), exist_ok=True)
    final_df.to_csv(DATASET_CSV, index=False, encoding="utf-8-sig")
    print(f"\n🎉 Dataset actualizado guardado en '{DATASET_CSV}'")
else:
    print("\n✅ No hubo actualizaciones, el dataset se mantiene igual.")

# ------------- SALIDA JSON (anidado por empresa) -------------
for empresa in empresas_data:
    nombre_empresa = empresa.get("nombre", "N/A")
    empresa["reseñas"] = reviews_by_company.get(nombre_empresa, [])

os.makedirs(os.path.dirname(SALIDA_JSON), exist_ok=True)
with open(SALIDA_JSON, "w", encoding="utf-8") as f:
    json.dump(empresas_data, f, ensure_ascii=False, indent=4, default=str)

print(f"📦 Archivo JSON guardado en '{SALIDA_JSON}'")



🔍 Verificando nuevas reseñas de Heymondo Seguros de Viaje (heymondo.es)...
🌐 Accediendo a: https://es.trustpilot.com/review/heymondo.es
✅ 20 reseñas nuevas en página 1. Continuando...
🌐 Accediendo a: https://es.trustpilot.com/review/heymondo.es?page=2
✅ 20 reseñas nuevas en página 2. Continuando...
🌐 Accediendo a: https://es.trustpilot.com/review/heymondo.es?page=3
✅ 20 reseñas nuevas en página 3. Continuando...
🌐 Accediendo a: https://es.trustpilot.com/review/heymondo.es?page=4
✅ 20 reseñas nuevas en página 4. Continuando...
🌐 Accediendo a: https://es.trustpilot.com/review/heymondo.es?page=5
✅ 20 reseñas nuevas en página 5. Continuando...

🔍 Verificando nuevas reseñas de Intermundial Seguros de viaje (intermundial.es)...
🌐 Accediendo a: https://es.trustpilot.com/review/intermundial.es
✅ 20 reseñas nuevas en página 1. Continuando...
🌐 Accediendo a: https://es.trustpilot.com/review/intermundial.es?page=2
✅ 20 reseñas nuevas en página 2. Continuando...
🌐 Accediendo a: https://es.trustpi

### Categoria 3

In [8]:
import time
import json
import pandas as pd
import os
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ------------- CONFIG -------------
DEBUG = False                 # True = imprime y guarda evidencia (screenshot + html)
MAX_PAGES = 5                 # máximo de páginas por empresa
EMPRESAS_JSON = 'datalake/1_LANDING_ZONE/trustpilot_empresas_categoria3.json'
DATASET_CSV   = 'datalake/1_LANDING_ZONE/dataset_categoria3.csv'
SALIDA_JSON   = 'datalake/1_LANDING_ZONE/reviews_trustpilot_empresas_categoria3.json'

# ------------- HELPERS -------------
def norm_text(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "").strip())

def parse_estrellas_from_alt(alt_txt: str):
    """
    'Valorada con 4 estrellas sobre 5' -> '4'
    """
    if not alt_txt:
        return "N/A"
    m = re.search(r"(\d)\s*estrellas", alt_txt, flags=re.I)
    return m.group(1) if m else "N/A"

def normalizar_dominio(pagina_web: str) -> str:
    """
    Espera 'www.midominio.es' o 'midominio.es'.
    Quita http(s):// si viene, y slashes al final.
    """
    if not pagina_web:
        return ""
    pw = pagina_web.strip()
    pw = re.sub(r"^https?://", "", pw, flags=re.I)
    pw = pw.strip("/")
    return pw

def extraer_review_uid(card):
    """
    Intenta extraer el id de la reseña desde el href del título: '/reviews/<id>'
    Devuelve el segmento <id> o '' si no existe.
    """
    try:
        a = card.find_element(By.CSS_SELECTOR, 'a[data-review-title-typography="true"]')
        href = a.get_attribute("href")
        if href:
            m = re.search(r"/reviews/([^/?#]+)", href)
            if m:
                return m.group(1)
    except:
        pass
    return ""

def aceptar_cookies_si_aparecen(driver, wait):
    """
    Cierra la barra de cookies de Trustpilot si aparece (botón 'Entendido' o similar).
    """
    try:
        btn = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, "//button[.//span[normalize-space(text())='Entendido'] or normalize-space(text())='Entendido']"))
        )
        btn.click()
        time.sleep(0.5)
        return True
    except:
        try:
            btn2 = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, "//button[normalize-space(text())='Aceptar']"))
            )
            btn2.click()
            time.sleep(0.5)
            return True
        except:
            return False

# ------------- CARGA EMPRESAS -------------
with open(EMPRESAS_JSON, 'r', encoding='utf-8') as f:
    empresas_data = json.load(f)

# ------------- CARGA / CREA DATASET -------------
if os.path.exists(DATASET_CSV):
    existing_df = pd.read_csv(DATASET_CSV)
else:
    existing_df = pd.DataFrame(columns=["id_reseña", "Fecha", "Título", "Contenido", "Empresa", "Calificación"])

existing_df['Fecha'] = pd.to_datetime(existing_df['Fecha'], errors='coerce')

# ------------- SELENIUM -------------
options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--window-size=1366,2400")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/118.0.5993.70 Safari/537.36"
)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 25)

resultados = []
reviews_by_company = {}

try:
    for empresa in empresas_data:
        nombre_empresa = empresa.get("nombre", "N/A")
        pagina_web = normalizar_dominio(empresa.get("pagina_web", ""))
        if not pagina_web:
            print(f"⚠️ Empresa sin 'pagina_web': {nombre_empresa}. Omitida.")
            continue

        print(f"\n🔍 Verificando nuevas reseñas de {nombre_empresa} ({pagina_web})...")

        # Histórico de esa empresa
        old_df = existing_df[existing_df["Empresa"] == nombre_empresa].copy()
        if not old_df.empty:
            old_df["Título"] = old_df["Título"].fillna("").str.strip()
            old_df["Fecha"] = pd.to_datetime(old_df["Fecha"], errors="coerce")

        reseñas_nuevas = []

        for page in range(1, MAX_PAGES + 1):
            review_url = f"https://es.trustpilot.com/review/{pagina_web}" + (f"?page={page}" if page > 1 else "")
            print(f"🌐 Accediendo a: {review_url}")
            driver.get(review_url)

            # Aceptar cookies si estorban
            aceptar_cookies_si_aparecen(driver, wait)

            # Espera a que al menos haya 1 card de reseña en la página
            try:
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-testid="service-review-card-v2"]')))
            except Exception as e:
                print(f"⚠️ No se encontraron cards en página {page}: {e}")
                if DEBUG:
                    ss = f"_debug_reviews_{nombre_empresa}_p{page}_nocards.png"
                    driver.save_screenshot(ss)
                    with open(f"_debug_reviews_{nombre_empresa}_p{page}_nocards.html", "w", encoding="utf-8") as fh:
                        fh.write(driver.page_source)
                    print(f"💾 Guardé evidencia sin cards: {ss}")
                break

            # Forzar render con scrolls suaves (contenido lazy)
            for _ in range(3):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1.2)

            # Cards reales (búsqueda global)
            cards = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid="service-review-card-v2"]')

            if DEBUG:
                print(f"🔎 Encontradas {len(cards)} cards en página {page}")
                try:
                    ss_name = f"_debug_reviews_{nombre_empresa}_p{page}.png"
                    html_name = f"_debug_reviews_{nombre_empresa}_p{page}.html"
                    driver.save_screenshot(ss_name)
                    with open(html_name, "w", encoding="utf-8") as fh:
                        fh.write(driver.page_source)
                    print(f"💾 Guardé screenshot y HTML: {ss_name} / {html_name}")
                except Exception as e:
                    print(f"⚠️ No pude guardar debug files: {e}")

            nuevas_esta_pagina = []

            for i, card in enumerate(cards, start=1):
                try:
                    # Fecha
                    try:
                        time_el = card.find_element(By.CSS_SELECTOR, 'time[datetime]')
                        fecha = pd.to_datetime(time_el.get_attribute("datetime"), errors='coerce')
                    except:
                        fecha = pd.NaT

                    # Título
                    try:
                        h2 = card.find_element(By.CSS_SELECTOR, 'h2[data-service-review-title-typography="true"]')
                        titulo = norm_text(h2.get_attribute("innerText") or h2.text)
                    except:
                        titulo = "N/A"

                    # Contenido
                    try:
                        p = card.find_element(By.CSS_SELECTOR, 'p[data-service-review-text-typography="true"]')
                        contenido = norm_text(p.get_attribute("innerText") or p.text)
                    except:
                        contenido = "Sin contenido"

                    # Calificación (solo número)
                    try:
                        img = card.find_element(By.CSS_SELECTOR, 'img[alt*="Valorada con"]')
                        estrellas = parse_estrellas_from_alt(img.get_attribute("alt"))
                    except:
                        estrellas = "N/A"

                    # UID de la review (opcional)
                    review_uid = extraer_review_uid(card)

                    if DEBUG and i <= 3:
                        print(f"  • Card #{i}: fecha={fecha}, estrellas={estrellas}, título='{titulo[:60]}', uid='{review_uid}'")

                    # DEDUP: Título + Fecha
                    existe = (
                        not old_df[
                            (old_df["Título"].str.strip() == titulo) &
                            (old_df["Fecha"] == fecha)
                        ].empty
                    )

                    if not existe:
                        nuevas_esta_pagina.append({
                            "Empresa": nombre_empresa,
                            "Título": titulo,
                            "Contenido": contenido,
                            "Fecha": fecha,
                            "Calificación": estrellas
                        })

                except Exception as e_card:
                    if DEBUG:
                        print(f"  • Aviso: card con error: {e_card}")
                    continue

            if nuevas_esta_pagina:
                reseñas_nuevas.extend(nuevas_esta_pagina)
                print(f"✅ {len(nuevas_esta_pagina)} reseñas nuevas en página {page}. Continuando...")
            else:
                print(f"🛑 No hay reseñas nuevas en página {page}. Deteniendo scraping para esta empresa.")
                break

        # Consolidar por empresa
        if reseñas_nuevas:
            df_nuevas = pd.DataFrame(reseñas_nuevas)
            df_nuevas['Fecha'] = pd.to_datetime(df_nuevas['Fecha'], errors='coerce')
            combined_df = pd.concat([old_df, df_nuevas], ignore_index=True)
            combined_df.drop_duplicates(subset=["Título", "Fecha"], keep='first', inplace=True)
            combined_df.sort_values(by="Fecha", ascending=False, inplace=True)
            combined_df.reset_index(drop=True, inplace=True)
            combined_df.fillna("", inplace=True)

            total = len(combined_df)
            combined_df["id_reseña"] = [f"{nombre_empresa.replace(' ', '')}_N{total - idx}" for idx in range(total)]

            reviews_by_company[nombre_empresa] = combined_df.to_dict(orient="records")
            resultados.append(combined_df)
        else:
            if not old_df.empty:
                reviews_by_company[nombre_empresa] = old_df.to_dict(orient="records")
            print(f"📭 No se detectaron nuevas reseñas para {nombre_empresa}.")

finally:
    driver.quit()

# ------------- SALIDA CSV -------------
if resultados:
    final_df = pd.concat([existing_df] + resultados, ignore_index=True)
    final_df.drop_duplicates(subset=["Título", "Fecha", "Empresa"], keep='first', inplace=True)
    final_df.fillna("", inplace=True)

    final_df["Fecha"] = pd.to_datetime(final_df["Fecha"], errors="coerce")
    final_df.sort_values(by=["Empresa", "Fecha"], ascending=[True, False], inplace=True)

    final_df["id_reseña"] = final_df.groupby("Empresa").cumcount(ascending=False) + 1
    final_df["id_reseña"] = final_df.apply(
        lambda row: f"{row['Empresa'].replace(' ', '')}_N{row['id_reseña']}", axis=1
    )

    final_df = final_df[["id_reseña", "Fecha", "Título", "Contenido", "Empresa", "Calificación"]]

    final_df["id_num"] = final_df["id_reseña"].str.extract(r'_N(\d+)').astype(int)
    final_df.sort_values(by=["Empresa", "id_num"], ascending=[True, False], inplace=True)
    final_df.drop(columns=["id_num"], inplace=True)

    os.makedirs(os.path.dirname(DATASET_CSV), exist_ok=True)
    final_df.to_csv(DATASET_CSV, index=False, encoding="utf-8-sig")
    print(f"\n🎉 Dataset actualizado guardado en '{DATASET_CSV}'")
else:
    print("\n✅ No hubo actualizaciones, el dataset se mantiene igual.")

# ------------- SALIDA JSON (anidado por empresa) -------------
for empresa in empresas_data:
    nombre_empresa = empresa.get("nombre", "N/A")
    empresa["reseñas"] = reviews_by_company.get(nombre_empresa, [])

os.makedirs(os.path.dirname(SALIDA_JSON), exist_ok=True)
with open(SALIDA_JSON, "w", encoding="utf-8") as f:
    json.dump(empresas_data, f, ensure_ascii=False, indent=4, default=str)

print(f"📦 Archivo JSON guardado en '{SALIDA_JSON}'")



🔍 Verificando nuevas reseñas de compramostucoche.es (www.compramostucoche.es)...
🌐 Accediendo a: https://es.trustpilot.com/review/www.compramostucoche.es
✅ 20 reseñas nuevas en página 1. Continuando...
🌐 Accediendo a: https://es.trustpilot.com/review/www.compramostucoche.es?page=2
✅ 20 reseñas nuevas en página 2. Continuando...
🌐 Accediendo a: https://es.trustpilot.com/review/www.compramostucoche.es?page=3
✅ 20 reseñas nuevas en página 3. Continuando...
🌐 Accediendo a: https://es.trustpilot.com/review/www.compramostucoche.es?page=4
✅ 20 reseñas nuevas en página 4. Continuando...
🌐 Accediendo a: https://es.trustpilot.com/review/www.compramostucoche.es?page=5
✅ 20 reseñas nuevas en página 5. Continuando...

🔍 Verificando nuevas reseñas de OcasionPlus (ocasionplus.com)...
🌐 Accediendo a: https://es.trustpilot.com/review/ocasionplus.com
✅ 20 reseñas nuevas en página 1. Continuando...
🌐 Accediendo a: https://es.trustpilot.com/review/ocasionplus.com?page=2
✅ 20 reseñas nuevas en página 2. C

### 2 - Unir los dataset

In [13]:
import pandas as pd
import json
from collections import OrderedDict

# Cargar los datasets CSV
df1 = pd.read_csv("datalake/1_LANDING_ZONE/dataset_categoria1.csv")
df2 = pd.read_csv("datalake/1_LANDING_ZONE/dataset_categoria2.csv")
df3 = pd.read_csv("datalake/1_LANDING_ZONE/dataset_categoria3.csv")  # Nuevo dataset

# Agregar columna 'categoria'
df1["categoria"] = "Banco"
df2["categoria"] = "Seguros de viaje"
df3["categoria"] = "Concesionario de autos"  # Nueva categoría

# Reordenar las columnas para que 'categoria' aparezca al inicio
def reorder_columns(df):
    return df[["categoria"] + [col for col in df.columns if col != "categoria"]]

df1 = reorder_columns(df1)
df2 = reorder_columns(df2)
df3 = reorder_columns(df3)

# Combinar todos los DataFrames
merged_csv = pd.concat([df1, df2, df3], ignore_index=True)

# Guardar el resultado en un nuevo archivo CSV
merged_csv.to_csv("datalake/1_LANDING_ZONE/dataset_reviews.csv", index=False, encoding="utf-8-sig")
print("CSV fusionado guardado como dataset_reviews.csv")

################### JSON ###################

# Cargar los archivos JSON de entrada
with open("datalake/1_LANDING_ZONE/reviews_trustpilot_empresas_categoria1.json", "r", encoding="utf-8") as f:
    data1 = json.load(f)
with open("datalake/1_LANDING_ZONE/reviews_trustpilot_empresas_categoria2.json", "r", encoding="utf-8") as f:
    data2 = json.load(f)
with open("datalake/1_LANDING_ZONE/reviews_trustpilot_empresas_categoria3.json", "r", encoding="utf-8") as f:
    data3 = json.load(f)

# Agregar la clave 'categoria' a cada empresa
for company in data1:
    company["categoria"] = "Bancos"
for company in data2:
    company["categoria"] = "Seguro de viajes"
for company in data3:
    company["categoria"] = "Concesionario de autos"

# Combinar las listas de empresas
merged_data = data1 + data2 + data3

# Reordenar las claves para que 'categoria' aparezca al inicio
ordered_merged_data = []
for company in merged_data:
    ordered_company = OrderedDict()
    ordered_company["categoria"] = company.get("categoria", "")
    for key in company:
        if key != "categoria":
            ordered_company[key] = company[key]
    ordered_merged_data.append(ordered_company)

# Guardar el JSON combinado en un nuevo archivo
with open("datalake/1_LANDING_ZONE/dataset_reviews.json", "w", encoding="utf-8") as f:
    json.dump(ordered_merged_data, f, ensure_ascii=False, indent=4)

print("JSON fusionado guardado como dataset_reviews.json")


CSV fusionado guardado como dataset_reviews.csv
JSON fusionado guardado como dataset_reviews.json


## 2 - Data Cleaning & Transformation


In [14]:
import json
import re
import unicodedata
from datetime import datetime, timezone
import pytz
import pandas as pd
from sqlalchemy import create_engine, text

# Definir rutas de archivos
input_file = "datalake/1_LANDING_ZONE/dataset_reviews.json"
output_file = "datalake/2_REFINED_ZONE/dataset_reviews_limpio.json"

# Configurar zona horaria de Bolivia (America/La_Paz)
bolivia_tz = pytz.timezone("America/La_Paz")

def clean_puntuacion(puntuacion):
    """
    Devuelve la puntuación como float (>0) o "" si no es válida.
    - Soporta float/int directamente.
    - Si es str, elimina 'TrustScore', cambia coma por punto y castea a float.
    """
    if puntuacion is None:
        return ""
    # Si ya es numérico
    if isinstance(puntuacion, (int, float)):
        try:
            val = float(puntuacion)
            return val if val > 0 else ""
        except Exception:
            return ""
    # Si es texto
    if isinstance(puntuacion, str):
        cleaned = puntuacion.replace("TrustScore", "").strip().replace(",", ".")
        # Si vienen otras palabras, intenta extraer el primer número
        m = re.search(r"(\d+(?:\.\d+)?)", cleaned)
        if m:
            try:
                val = float(m.group(1))
                return val if val > 0 else ""
            except ValueError:
                return ""
        try:
            val = float(cleaned)
            return val if val > 0 else ""
        except ValueError:
            return ""
    # Cualquier otro tipo
    return ""

def convert_datetime(dt_str):
    """
    Convierte a zona Bolivia y retorna (fecha_local, hora_local, iso_bolivia).
    Acepta strings ISO como '2025-11-08 13:51:34+00:00' o '2025-11-08T13:51:34+00:00'.
    Si es naive, asume UTC.
    """
    if not dt_str:
        return "", "", ""
    s = str(dt_str)
    # Normaliza separador
    if " " in s and "T" not in s:
        s = s.replace(" ", "T", 1)
    try:
        dt = datetime.fromisoformat(s)
    except Exception:
        return "", "", ""
    # Si es naive, asumimos UTC
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    try:
        dt_bolivia = dt.astimezone(bolivia_tz)
        fecha_local = dt_bolivia.strftime("%Y-%m-%d")
        hora_local = dt_bolivia.strftime("%H:%M:%S")
        return fecha_local, hora_local, dt_bolivia.isoformat()
    except Exception:
        return "", "", ""

def clean_text(text):
    """
    Minúsculas, quita acentos, remueve caracteres no alfanuméricos (y emojis),
    colapsa espacios.
    """
    if text is None:
        return ""
    text = str(text).lower()
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Cargar el archivo JSON de entrada
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Procesar cada empresa en el dataset
for company in data:
    # Asegurar que los campos de nivel empresa sean textos (cuando apliquen)
    for key in ["categoria", "nombre", "ubicacion", "pagina_web"]:
        if key in company and company[key] is not None:
            company[key] = str(company[key]).strip()

    # Si la ubicación es "N/A", reemplazar por "sin ubicacion"
    if company.get("ubicacion", "").strip().upper() == "N/A":
        company["ubicacion"] = "sin ubicacion"

    # Limpiar el campo 'puntuacion' (ahora puede ser float)
    if "puntuacion" in company:
        company["puntuacion"] = clean_puntuacion(company["puntuacion"])

    # Procesar las reseñas
    if "reseñas" in company and isinstance(company["reseñas"], list):
        for review in company["reseñas"]:
            # Normaliza campos base a string
            for key in ["id_reseña", "Título", "Contenido", "Empresa"]:
                if key in review and review[key] is not None:
                    review[key] = str(review[key]).strip()

            # Limpiar Título y Contenido
            if "Título" in review:
                review["Título"] = clean_text(review["Título"])
            if "Contenido" in review:
                review["Contenido"] = clean_text(review["Contenido"])

            # Convertir fecha a zona Bolivia
            if "Fecha" in review and review["Fecha"]:
                fecha_local, hora_local, iso_bolivia = convert_datetime(review["Fecha"])
                # Usa minúsculas para consistencia con tu dashboard
                review["fecha_local"] = fecha_local
                review["hora_local"] = hora_local
                # Si te sirve conservar el ISO local, puedes guardarlo en otro campo:
                # review["fecha_iso_bolivia"] = iso_bolivia

            # Validar "Calificación": entero > 0, tolerando 'N/A', '', etc.
            if "Calificación" in review:
                try:
                    cal = int(str(review["Calificación"]).strip())
                    review["Calificación"] = cal if cal > 0 else ""
                except Exception:
                    review["Calificación"] = ""

            # Eliminar campos no requeridos
            review.pop("Fecha", None)     # ya convertida a fecha_local/hora_local
            review.pop("Empresa", None)   # evitar redundancia

# Guardar los datos limpios en formato JSON
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4, default=str)

print(f"Datos limpios y validados guardados en {output_file}")


Datos limpios y validados guardados en datalake/2_REFINED_ZONE/dataset_reviews_limpio.json


#### Subir en la base de datos

## 3 - Data Lake Architecture

In [15]:
#!streamlit run datalake/3_CONSUMPTION_ZONE/app.py


Windows

In [16]:
# cd C:\Users\DANIEL\Downloads\M7_PROYECTO_DANIEL_SANCHEZ\M7_PROYECTO_DANIEL_SANCHEZ
#streamlit run datalake\3_CONSUMPTION_ZONE\app.py
